# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [1]:
# Your code here
import pandas as pd
import sqlite3
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [5]:
# Your code here
query = '''
    SELECT employees.firstName, employees.lastName
    FROM employees
    JOIN offices  ON employees.officeCode = offices.officeCode
    WHERE offices.city = 'Boston';
'''

pd.read_sql(query, conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [9]:
# Your code here
query = '''
    SELECT offices.officeCode, offices.city, COUNT(employees.employeeNumber) AS numEmployees
    FROM offices
    LEFT JOIN employees ON offices.officeCode = employees.officeCode
    GROUP BY offices.officeCode, offices.city
    HAVING numEmployees = 0;
    
'''

pd.read_sql(query, conn)

,officeCode,city,numEmployees
0,27,Boston,0


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [13]:
"""
Question 1: Are there customers who have never placed an order?
"""

# Your code here
query = '''
    SELECT customers.customerNumber, customers.customerName
    FROM customers
    LEFT JOIN orders ON customers.customerNumber = orders.customerNumber
    WHERE orders.orderNumber IS NULL;
'''

pd.read_sql(query, conn)

,customerNumber,customerName
0,125,Havel & Zbyszek Co
1,168,American Souvenirs Inc
2,169,Porto Imports Co.
3,206,"Asian Shopping Network, Co"
4,223,Natürlich Autos
5,237,ANG Resellers
6,247,Messner Shopping Network
7,273,"Franken Gifts, Co"
8,293,BG&E Collectables
9,303,Schuyler Imports


In [19]:
"""
Question 2:What are the top 10 orders made?
"""

# Your code here
query = '''
    SELECT orders.orderNumber, SUM(orderdetails.quantityOrdered * orderdetails.priceEach) As totalValue
    FROM orders
    JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
    GROUP BY orders.orderNumber
    ORDER BY totalValue DESC
    LIMIT 10;
'''
pd.read_sql(query, conn)


,orderNumber,totalValue
0,10165,67392.85
1,10287,61402.00
2,10310,61234.67
3,10212,59830.55
4,10207,59265.14
5,10127,58841.35
6,10204,58793.53
7,10126,57131.92
8,10222,56822.65
9,10142,56052.56


In [21]:
"""
Question 3:Who are the top 5 customer by orders made?
"""

# Your code here
query = '''
    SELECT customers.customerNumber, customers.customerName, SUM(orderdetails.quantityOrdered * orderdetails.priceEach) AS totalSpent
    FROM customers
    JOIN orders ON customers.customerNumber = orders.customerNumber
    JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
    GROUP BY customers.customerNumber, customers.customerName
    ORDER BY totalSpent DESC
    LIMIT 5;
'''
pd.read_sql(query, conn)

,customerNumber,customerName,totalSpent
0,141,Euro+ Shopping Channel,820689.54
1,124,Mini Gifts Distributors Ltd.,591827.34
2,114,"Australian Collectors, Co.",180585.07
3,151,Muscle Machine Inc,177913.95
4,119,La Rochelle Gifts,158573.12


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [23]:
# Your code here
query = '''
    SELECT employees.firstName, employees.lastName, products.productName
    FROM employees
    JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
    JOIN orders ON customers.customerNumber = orders.customerNumber
    JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
    JOIN products ON orderdetails.productCode = products.productCode
    ORDER BY employees.lastName, employees.firstName, products.productName;
'''
pd.read_sql(query, conn)

,firstName,lastName,productName
0,Loui,Bondur,18th century schooner
1,Loui,Bondur,1900s Vintage Bi-Plane
2,Loui,Bondur,1900s Vintage Tri-Plane
3,Loui,Bondur,1903 Ford Model A
4,Loui,Bondur,1903 Ford Model A
...,...,...,...
2991,George,Vanauf,The Titanic
2992,George,Vanauf,The Titanic
2993,George,Vanauf,The USS Constitution Ship
2994,George,Vanauf,The USS Constitution Ship


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [27]:
# Your code here
query = '''
    SELECT employees.firstName, employees.lastName, SUM(orderdetails.quantityOrdered) AS totalProductsSold
    FROM employees
    JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
    JOIN orders ON customers.customerNumber = orders.customerNumber
    JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
    GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
    ORDER BY employees.lastName, employees.firstName;
'''
pd.read_sql(query, conn)

,firstName,lastName,totalProductsSold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [41]:
# Your code here
query = '''
    SELECT employees.firstName,employees.lastName, COUNT(DISTINCT orderdetails.productCode) AS uniqueProductsSold
    FROM employees
    JOIN customers ON employees.employeeNumber = customers.salesRepEmployeeNumber
    JOIN orders ON customers.customerNumber = orders.customerNumber
    JOIN orderdetails ON orders.orderNumber = orderdetails.orderNumber
    GROUP BY employees.employeeNumber, employees.firstName, employees.lastName
    HAVING COUNT(DISTINCT orderdetails.productCode) > 200
    ORDER BY employees.lastName, employees.firstName;

'''
pd.read_sql(query, conn)

,firstName,lastName,uniqueProductsSold


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!